In [42]:
import os
import sys
import pandas
import numpy
import scipy

import findspark
findspark.init("/usr/local/spark/spark-2.2.0-bin-hadoop2.7/")

import pyspark
from pyspark.sql.window import Window
import pyspark.sql.functions as func

from pyspark.rdd import reduce
from pyspark.sql.types import DoubleType
from pyspark.ml.feature import VectorAssembler
import pyspark.mllib.linalg.distributed
from pyspark.mllib.linalg.distributed import RowMatrix, DenseMatrix
from pyspark.ml.linalg import SparseVector, VectorUDT, Vector, Vectors
from pyspark.mllib.stat import Statistics

In [2]:
conf = pyspark.SparkConf().setMaster("local[*]").set("spark.driver.memory", "4G").set("spark.executor.memory", "4G")
sc = pyspark.SparkContext(conf=conf)
spark = pyspark.sql.SparkSession(sc)

In [3]:
spark.version

'2.2.0'

In [4]:
file_name = "/Users/simondi/PHD/data/data/target_infect_x/query_data/cells_sample_10_normalized_cut_100.tsv"
df = spark.read.csv(path=file_name, sep="\t", header='true')

In [5]:
old_cols = df.columns
new_cols = list(map(lambda x: x.replace(".", "_"), old_cols))

df = reduce(
  lambda data, idx: data.withColumnRenamed(old_cols[idx], new_cols[idx]),
  range(len(new_cols)), df)

for i, x in enumerate(new_cols):
    if x.startswith("cells") or x.startswith("perin") or x.startswith("nucl"):
        df = df.withColumn(x, df[x].cast("double"))

df = df.fillna(0)

In [98]:
feature_columns = [x for x in df.columns if (x.startswith("cells") or x.startswith("perin") or x.startswith("nucl")) and not x.startswith("cells_children_invasomes_count")]
assembler = VectorAssembler(inputCols=feature_columns, outputCol='features')
data = assembler.transform(df)

In [99]:
data

DataFrame[study: string, pathogen: string, library: string, design: string, replicate: string, plate: string, well: string, gene: string, sirna: string, well_type: string, image_idx: string, object_idx: string, cells_areashape_area: double, cells_areashape_eccentricity: double, cells_areashape_extent: double, cells_areashape_formfactor: double, cells_areashape_majoraxislength: double, cells_areashape_minoraxislength: double, cells_areashape_perimeter: double, cells_children_bacteria_count: double, cells_children_invasomes_count: double, cells_location_center_x: double, cells_location_center_y: double, cells_neighbors_anglebetweenneighbors_2: double, cells_neighbors_firstclosestobjectnumber_2: double, cells_neighbors_firstclosestxvector_2: double, cells_neighbors_firstclosestyvector_2: double, cells_neighbors_numberofneighbors_2: double, cells_neighbors_secondclosestobjectnumber_2: double, cells_neighbors_secondclosestxvector_2: double, cells_neighbors_secondclosestyvector_2: double, ce

In [123]:
rdd = data.select(feature_columns).rdd.map(list)
X = RowMatrix(rdd)

In [101]:
X.numCols()

19

In [143]:
summary = Statistics.colStats(rdd)
means = summary.mean()
means

array([-0.06635729, -0.11428057,  0.11343067,  0.05040407, -0.10184821,
       -0.00780856, -0.07642234, -0.1275254 , -0.02282229, -0.06262571,
        0.12484165,  0.05149121,  0.0084862 ,  0.06069586,  0.14847159,
        0.03277408, -0.08037804,  0.00642936,  0.04285231])

In [145]:
def svd(X, comps):
    svd = X.computeSVD(X.numCols(), computeU=False)
    s = svd.s.toArray()
    V = svd.V.toArray().T
    var = numpy.dot(s[comps:], s[comps:])
    return s[:comps], V[:comps], var

In [146]:
def tilde(X, psi_sqrt, n_sqrt):
    norm = psi_sqrt * n_sqrt
    Xtilde = RowMatrix(X.rows.map(lambda x: x / norm))
    return Xtilde

In [144]:
X = RowMatrix(X.rows.map(lambda x: x - means))

In [148]:
iter = 0
DELTA = 1e-12
MAX_ITER = 100

N, P = X.numRows(), X.numCols()
ll = old_ll = -numpy.inf
psi = numpy.ones(P, dtype=numpy.float64)
var = summary.variance()
nsqrt = numpy.sqrt(N)

n_components = 2
llconst = P * numpy.log(2. * numpy.pi) + n_components

In [149]:
for i in range(5):
    sqrt_psi = numpy.sqrt(psi) + DELTA
    s, V, unexp_var = svd(tilde(X, sqrt_psi, nsqrt), 2)
    s = s ** 2
    W = numpy.sqrt(numpy.maximum(s - 1., 0.))[:, numpy.newaxis] * V
    W *= sqrt_psi
    ll = llconst + numpy.sum(numpy.log(s))
    ll += unexp_var + numpy.sum(numpy.log(psi))
    ll *= -N / 2.
    psi = numpy.maximum(var - numpy.sum(W ** 2, axis=0), DELTA)
    if numpy.abs(ll - old_ll) < 0.001:
        break
    old_ll = ll

In [150]:
print(psi)

[ 0.09317412  1.15113511  0.87519844  0.87889341  0.25706024  0.32922078
  0.25494487  0.46813424  0.0978511   0.98867626  0.94668337  0.03096463
  0.73542173  0.8975675   0.6086216   0.03500784  0.87084055  0.74302272
  0.02427419]


In [84]:
Ih = numpy.eye(len(W))
Wpsi = W / psi
cov_z = scipy.linalg.inv(Ih + numpy.dot(Wpsi, W.T))
tmp = numpy.dot(Wpsi.T, cov_z)

In [154]:
print(cov_z)

[[  6.98387235e-03   4.33070002e-05]
 [  4.33070002e-05   6.40090401e-02]]


In [156]:
print(Wpsi)

[[ -9.68451280e-01  -4.28387809e-02  -6.09688956e-02  -2.25291717e-01
   -3.89265464e-01  -1.42013509e-01  -3.05073712e-03   1.48088216e-01
    1.98849471e-36   1.01051829e+01  -6.86097934e-02   7.27866244e-03
    3.93981142e+01  -3.88149420e-02   6.11372369e-02   8.99442604e-02
    3.22553148e+01   1.13557979e-02  -4.12123744e-02   5.01222871e+01]
 [  9.08607706e+00  -1.15169464e-01  -1.59840573e-01  -4.23621959e-01
    2.86601506e+00   2.50557131e+00   2.92465532e+00   1.31105496e-01
    0.00000000e+00   7.02061859e-01  -6.54677157e-02   2.72602163e-02
    4.86192461e-01   1.11598614e-01   9.93284647e-02   9.21675584e-01
   -3.11499239e-01  -2.03931577e-01   2.33386539e-01   3.13683528e-02]]


In [152]:
print(psi)

[ 0.09317412  1.15113511  0.87519844  0.87889341  0.25706024  0.32922078
  0.25494487  0.46813424  0.0978511   0.98867626  0.94668337  0.03096463
  0.73542173  0.8975675   0.6086216   0.03500784  0.87084055  0.74302272
  0.02427419]


In [85]:
tmp_dense = DenseMatrix(numRows=tmp.shape[0], numCols=tmp.shape[1], values=tmp.flatten())

In [86]:
Xtran = X.multiply(tmp_dense)

In [87]:
Xtran.rows.take(5)

[DenseVector([0.5513, 0.2527]),
 DenseVector([-0.0769, -0.4427]),
 DenseVector([0.8807, 0.1603]),
 DenseVector([0.6999, 0.4691]),
 DenseVector([-0.7997, -0.2368])]